#### El primer caso es importar las librerías necesarias. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#### Creo el data frame en base al csv

In [2]:
tweets = pd.read_csv('csv/train.csv', encoding='latin-1')
tweets.tail()

,id,keyword,location,text,target
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1
7612,10873,NaN,NaN,The Latest: More Homes Razed by Northern Calif...,1


##### Convierto el target a boolean

In [3]:
tweets['target'] = tweets['target'].astype(bool)

In [4]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   bool  
dtypes: bool(1), int64(1), object(3)
memory usage: 245.5+ KB


In [5]:
serieDeTweets = tweets['text'].str.split()
type(serieDeTweets)

pandas.core.series.Series

#### Veo como esta formada esta serie

In [6]:
serieDeTweets

0       [Our, Deeds, are, the, Reason, of, this, #eart...
1          [Forest, fire, near, La, Ronge, Sask., Canada]
2       [All, residents, asked, to, 'shelter, in, plac...
3       [13,000, people, receive, #wildfires, evacuati...
4       [Just, got, sent, this, photo, from, Ruby, #Al...
                              ...                        
7608    [Two, giant, cranes, holding, a, bridge, colla...
7609    [@aria_ahrary, @TheTawniest, The, out, of, con...
7610    [M1.94, [01:04, UTC]?5km, S, of, Volcano, Hawa...
7611    [Police, investigating, after, an, e-bike, col...
7612    [The, Latest:, More, Homes, Razed, by, Norther...
Name: text, Length: 7613, dtype: object

#### mi objetivo es separar todas las palabras.

In [7]:
listaDePalabras = []
for palabra in serieDeTweets:
    listaDePalabras.extend(palabra)

In [8]:
listaDePalabras

['Our',
 'Deeds',
 'are',
 'the',
 'Reason',
 'of',
 'this',
 '#earthquake',
 'May',
 'ALLAH',
 'Forgive',
 'us',
 'all',
 'Forest',
 'fire',
 'near',
 'La',
 'Ronge',
 'Sask.',
 'Canada',
 'All',
 'residents',
 'asked',
 'to',
 "'shelter",
 'in',
 "place'",
 'are',
 'being',
 'notified',
 'by',
 'officers.',
 'No',
 'other',
 'evacuation',
 'or',
 'shelter',
 'in',
 'place',
 'orders',
 'are',
 'expected',
 '13,000',
 'people',
 'receive',
 '#wildfires',
 'evacuation',
 'orders',
 'in',
 'California',
 'Just',
 'got',
 'sent',
 'this',
 'photo',
 'from',
 'Ruby',
 '#Alaska',
 'as',
 'smoke',
 'from',
 '#wildfires',
 'pours',
 'into',
 'a',
 'school',
 '#RockyFire',
 'Update',
 '=>',
 'California',
 'Hwy.',
 '20',
 'closed',
 'in',
 'both',
 'directions',
 'due',
 'to',
 'Lake',
 'County',
 'fire',
 '-',
 '#CAfire',
 '#wildfires',
 '#flood',
 '#disaster',
 'Heavy',
 'rain',
 'causes',
 'flash',
 'flooding',
 'of',
 'streets',
 'in',
 'Manitou,',
 'Colorado',
 'Springs',
 'areas',
 "I'm

#### creo una serie en base a esto

In [9]:
palabras = pd.Series(listaDePalabras)
palabras

0                            Our
1                          Deeds
2                            are
3                            the
4                         Reason
                   ...          
113456                  Wildfire
113457                         -
113458                       ABC
113459                      News
113460    http://t.co/YmY4rSkQ3d
Length: 113461, dtype: object

#### Cuales son los mas repetidos


In [10]:
palabras.value_counts()

the               2575
a                 1845
to                1805
in                1757
of                1722
                  ... 
@ZacB_               1
sophistication       1
PPC                  1
Halifax              1
@chriscesq           1
Length: 31924, dtype: int64

#### me armo unos dataframe separando si los tweets son veraderos o no, luego sigo los mismos pasos para generar una serie de palabras.

In [11]:
tweetsVerdaderos = tweets.loc[tweets.target == True]
tweetsFalsos = tweets.loc[tweets.target == False]
serieDeTweetsVerdaderos = tweetsVerdaderos['text'].str.split()
serieDeTweetsFalsos = tweetsFalsos['text'].str.split()

listaDePalabrasVerdaderas = []
for palabra in serieDeTweetsVerdaderos:
    listaDePalabrasVerdaderas.extend(palabra)
    
listaDePalabrasFalsas = []
for palabra in serieDeTweetsFalsos:
    listaDePalabrasFalsas.extend(palabra)    

palabrasVerdaderas = pd.Series(listaDePalabrasVerdaderas)
palabrasFalsas = pd.Series(listaDePalabrasFalsas)

In [13]:
palabrasVerdaderas.value_counts()

the                       1051
in                        1037
of                         888
a                          730
to                         706
                          ... 
'casing                      1
@brianboru67                 1
park!                        1
http://t.co/aEtgbxm1pL       1
#OpSeaWorld                  1
Length: 16150, dtype: int64

In [14]:
palabrasFalsas.value_counts()

the                       1524
a                         1115
to                        1099
I                          938
and                        840
                          ... 
Activity                     1
weather.                     1
http://t.co/nyv51681uE       1
#radio                       1
Hutch.                       1
Length: 20560, dtype: int64